# Praktikum Pengantar Pembelajaran Mesin


---
## Bab 3. Klasifikasi K-Nearest Neighbor (KNN)


Tuliskan Nama, NIM, dan kelas Anda:

Nama  :Ghina Tri Fadilah

NIM   :215150207111073

Kelas :PPM-A

### 1) Import Data

Unduh dataset yang akan digunakan pada praktikum kali ini. Anda dapat menggunakan aplikasi wget untuk mendowload dataset dan menyimpannya dalam Google Colab. Jalankan cell di bawah ini untuk mengunduh dataset

In [ ]:
! wget https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv

--2023-03-16 02:04:32--  https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3975 (3.9K) [text/plain]
Saving to: ‘iris.csv’

iris.csv            100%[===================>]   3.88K  --.-KB/s    in 0s      

2023-03-16 02:04:33 (45.8 MB/s) - ‘iris.csv’ saved [3975/3975]



Setelah dataset berhasil diunduh, langkah berikutnya adalah membaca dataset dengan memanfaatkan fungsi **readcsv** dari library pandas. Lakukan pembacaan berkas csv ke dalam dataframe dengan nama **data** menggunakan fungsi **readcsv**. Jangan lupa untuk melakukan import library pandas terlebih dahulu


In [ ]:
import pandas as pd
import numpy as np
data = pd.read_csv('iris.csv')



Cek isi dataset Anda dengan menggunakan perintah **head()**

In [ ]:
data.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


## 2) Membagi data menjadi data latih dan data uji

Metode pembelajaran mesin memerlukan dua jenis data :


1.   Data latih : Digunakan untuk proses training metode klasifikasi
2.   Data uji : Digunakan untuk proses evaluasi metode klasifikasi

Data uji dan data latih perlu dibuat terpisah (mutualy exclusive) agar hasil evaluasi lebih akurat.

Data uji dan data latih dapat dibuat dengan cara membagi dataset dengan rasio tertentu, misalnya 80% data latih dan 20% data uji.

Library Scikit-learn memiliki fungsi [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) pada modul **model_selection** untuk membagi dataset menjadi data latih dan data uji. Bagilah dataset anda menjadi dua, yaitu **data_latih** dan **data_uji**.


In [ ]:
from sklearn.model_selection import train_test_split
data_latih,data_uji = train_test_split(data,test_size=0.2)

Tampilkan banyaknya data pada **data_latih** dan **data_uji**. Seharusnya **data_latih** terdiri dari 120 data, dan **data_uji** terdiri dari 30 data

In [ ]:
print(data_uji.shape[0])
print(data_latih.shape[0])

30
120


Pisahkan label/kategori dari data latih dan data uji menjadi variabel tersendiri. Beri nama variabelnya **label_latih** dan **label_uji**

In [ ]:
label_latih = data_latih.pop('variety')
label_uji = data_uji.pop('variety')

In [ ]:
data_uji.head

<bound method NDFrame.head of      sepal.length  sepal.width  petal.length  petal.width
54            6.5          2.8           4.6          1.5
15            5.7          4.4           1.5          0.4
51            6.4          3.2           4.5          1.5
18            5.7          3.8           1.7          0.3
6             4.6          3.4           1.4          0.3
10            5.4          3.7           1.5          0.2
0             5.1          3.5           1.4          0.2
42            4.4          3.2           1.3          0.2
16            5.4          3.9           1.3          0.4
24            4.8          3.4           1.9          0.2
50            7.0          3.2           4.7          1.4
99            5.7          2.8           4.1          1.3
72            6.3          2.5           4.9          1.5
141           6.9          3.1           5.1          2.3
97            6.2          2.9           4.3          1.3
94            5.6          2.7           4

## 3) Menghitung jarak euclidean

Tahapan awal dari algoritma KNN adalah perhitungan jarak. Salah satu metode perhitungan jarak yang bisa digunakan adalah jarak Euclidean. Buatlah fungsi bernama **jarakEu** yang berfungsi menghitung jarak euclidean dari dua buah vektor (tanpa kelas)

In [ ]:
def jarakEu(data1,data2):
  jarak = np.square(data1-data2)
  jarak = np.sum(jarak)
  return np.sqrt(jarak)

Uji fungsi **jarakEu** untuk menghitung jarak antara data latih pertama dengan data uji pertama.

In [ ]:
jarak = jarakEu(data_latih.iloc[0],data_uji.iloc[0])
print(jarak)

3.54259791678367


## 4) Algoritma KNN

Implementasikan algoritma KNN dengan tahapan-tahapan berikut :
1. Hitung jarak antara data uji dengan setiap data latih.
2. Cari *k* data latih dengan jarak terkecil.
3. Tentukan kelas dari *k* data latih tersebut.
4. Kelas data uji ditentukan dari mayoritas kelas *k* data latih.

Buatlah fungsi dengan nama **knn** yang menerima input berupa *k*, sekumpulan data latih dan labelnya, serta sebuah data uji

In [ ]:
from collections import Counter
def knn(k,datalatih,labellatih,datauji):
  jarak = np.array([jarakEu(datalatih.iloc[x],datauji)
  for x in range(datalatih.shape[0])])
  indeks_k_minimum = jarak.argsort()[:k]
  k_kelas = labellatih.iloc[indeks_k_minimum].to_numpy()
  counter = Counter(k_kelas)
  kelas_uji = counter.most_common(1)[0][0]
  return kelas_uji

Lakukan pengujian fungsi **knn** untuk menentukan kelas dari data uji pertama

In [ ]:
hasil = knn(3, data_latih, label_latih,data_uji.iloc[0])
print(hasil)

Versicolor


## 5) KNN pada Scikit-learn

Library scikit-learn memiliki algoritma KNN yang siap untuk digunakan. Sebagai pengenalan, Anda akan melakukan klasifikasi data yang sama menggunakan KNN yang tersedia pada library scikit-learn

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
KKN = KNeighborsClassifier(n_neighbors=3)
KKN.fit(data_latih,label_latih)
kelas = KKN.predict([data_uji.iloc[0]])
print(kelas)

['Versicolor']


/usr/local/lib/python3.9/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


## TUGAS
1. Buatlah fungsi bernama **jarakCity** yang berfungsi menghitung jarak City Block antara dua vektor.
2. Buatlah sebuah fungsi bernama **knn_multi** yang dapat menentukan kelas dari **semua** data uji (Fungsi **knn** yang sudah Anda buat hanya dapat menentukan kelas dari sebuah data uji. Anda dapat memanfaatkan fungsi **knn** yang telah ada
3. Buatlah fungsi bernama **cek_hasil** yang berfungsi membandingkan hasil klasifikasi dengan label sebenarnya. Fungsi ini menerima input 2 buah list, yaitu list label hasil klasifikasi dan list label data uji yang sebenarnya. Output dari fungsi ini adalah integer, yang menunjukkan berapa banyak label hasil klasifikasi yang sama dengan label data uji sebenarnya.
4. Jalankan fungsi **knn_multi** dengan nilai k=3, k=7,k=9 dan k=13. Amati hasil dari fungsi **cek_hasil** dengan nilai k tersebut. Apa kesimpulan Anda?

In [ ]:
#jawaban soal 1
def jarakCity(data1, data2):
  jarak = np.abs(data1 - data2)
  return np.sum(jarak)

In [ ]:
jarak = jarakCity(data_latih.iloc[0], data_uji.iloc[0])
jarak

6.1

In [ ]:
from scipy.spatial.distance import cityblock
print(cityblock(data_latih.iloc[0], data_uji.iloc[0]))

6.1


In [ ]:
def knn_Manhattan(k, data_latih, label_latih, data_uji):
  jarak = np.array([jarakCity(data_latih.iloc[x], data_uji) for x in range(data_latih.shape[0])])
  indeks_k_minimum = jarak.argsort()[:k]
  k_kelas = label_latih.iloc[indeks_k_minimum].to_numpy()
  counter = Counter(k_kelas)
  kelas_uji_mt = counter.most_common(1)[0][0]
  return kelas_uji_mt

In [ ]:
#jawaban soal 2
def knn_multi(k, datalatih, labellatih, datauji):
  kelas_hasil = []
  for i in range(datauji.shape[0]):
    kelas_hasil.append(knn_Manhattan(k, datalatih, labellatih, datauji.iloc[i]))
  return kelas_hasil

In [ ]:
label_uji

54     Versicolor
15         Setosa
51     Versicolor
18         Setosa
6          Setosa
10         Setosa
0          Setosa
42         Setosa
16         Setosa
24         Setosa
50     Versicolor
99     Versicolor
72     Versicolor
141     Virginica
97     Versicolor
94     Versicolor
9          Setosa
136     Virginica
55     Versicolor
38         Setosa
126     Virginica
63     Versicolor
28         Setosa
106     Virginica
45         Setosa
83     Versicolor
33         Setosa
75     Versicolor
113     Virginica
19         Setosa
Name: variety, dtype: object

In [ ]:
hasill = knn_multi(3, data_latih, label_latih, data_uji)
hasill

['Versicolor',
 'Setosa',
 'Versicolor',
 'Setosa',
 'Setosa',
 'Setosa',
 'Setosa',
 'Setosa',
 'Setosa',
 'Setosa',
 'Versicolor',
 'Versicolor',
 'Virginica',
 'Virginica',
 'Versicolor',
 'Versicolor',
 'Setosa',
 'Virginica',
 'Versicolor',
 'Setosa',
 'Virginica',
 'Versicolor',
 'Setosa',
 'Versicolor',
 'Setosa',
 'Virginica',
 'Setosa',
 'Versicolor',
 'Virginica',
 'Setosa']

In [ ]:
#jawaban soal 3
def cek_hasil(datahasil, datauji) :
  nilai = 0;
  for k in range(datauji.shape[0]):
    if datahasil[k]==datauji.iloc[k]:
      nilai = nilai+1

  return nilai

In [ ]:
nilai = cek_hasil(hasill, label_uji)
nilai

27

In [ ]:
#jawaban soal 4
k3 = knn_multi(3, data_latih, label_latih, data_uji)
k3

['Versicolor',
 'Setosa',
 'Versicolor',
 'Setosa',
 'Setosa',
 'Setosa',
 'Setosa',
 'Setosa',
 'Setosa',
 'Setosa',
 'Versicolor',
 'Versicolor',
 'Virginica',
 'Virginica',
 'Versicolor',
 'Versicolor',
 'Setosa',
 'Virginica',
 'Versicolor',
 'Setosa',
 'Virginica',
 'Versicolor',
 'Setosa',
 'Versicolor',
 'Setosa',
 'Virginica',
 'Setosa',
 'Versicolor',
 'Virginica',
 'Setosa']

In [ ]:
print('Cek hasil : ', cek_hasil(k3, label_uji))

Cek hasil :  27


In [ ]:
k7 = knn_multi(7, data_latih, label_latih, data_uji)
k7

['Versicolor',
 'Setosa',
 'Versicolor',
 'Setosa',
 'Setosa',
 'Setosa',
 'Setosa',
 'Setosa',
 'Setosa',
 'Setosa',
 'Versicolor',
 'Versicolor',
 'Virginica',
 'Virginica',
 'Versicolor',
 'Versicolor',
 'Setosa',
 'Virginica',
 'Versicolor',
 'Setosa',
 'Virginica',
 'Versicolor',
 'Setosa',
 'Versicolor',
 'Setosa',
 'Virginica',
 'Setosa',
 'Versicolor',
 'Virginica',
 'Setosa']

In [ ]:
print('Cek hasil : ', cek_hasil(k7, label_uji))

Cek hasil :  27


In [ ]:
k9 = knn_multi(9, data_latih, label_latih, data_uji)
k9

['Versicolor',
 'Setosa',
 'Versicolor',
 'Setosa',
 'Setosa',
 'Setosa',
 'Setosa',
 'Setosa',
 'Setosa',
 'Setosa',
 'Versicolor',
 'Versicolor',
 'Virginica',
 'Virginica',
 'Versicolor',
 'Versicolor',
 'Setosa',
 'Virginica',
 'Versicolor',
 'Setosa',
 'Virginica',
 'Versicolor',
 'Setosa',
 'Versicolor',
 'Setosa',
 'Virginica',
 'Setosa',
 'Versicolor',
 'Virginica',
 'Setosa']

In [ ]:
print('Cek hasil : ', cek_hasil(k9, label_uji))

Cek hasil :  27


In [ ]:
k13 = knn_multi(13, data_latih, label_latih, data_uji)
k13

['Versicolor',
 'Setosa',
 'Versicolor',
 'Setosa',
 'Setosa',
 'Setosa',
 'Setosa',
 'Setosa',
 'Setosa',
 'Setosa',
 'Versicolor',
 'Versicolor',
 'Virginica',
 'Virginica',
 'Versicolor',
 'Versicolor',
 'Setosa',
 'Virginica',
 'Versicolor',
 'Setosa',
 'Virginica',
 'Versicolor',
 'Setosa',
 'Versicolor',
 'Setosa',
 'Virginica',
 'Setosa',
 'Versicolor',
 'Virginica',
 'Setosa']

In [ ]:
print('Cek hasil : ', cek_hasil(k13, label_uji))

Cek hasil :  27


Penjelasan jawaban soal nomor 4

fungsi knn_multi adalah mengulang fungsi knn_manhattan yang akan mendapatkan prediksi kelas dari perhitungan jarak (manhattan style) antara 1 data uji dengan setiap data latih, sehingga knn_multi akan mendapatkan prediksi kelas dari setiap baris dari data uji.(total 30 baris).
Dari hasil percobaan di nomor 4, saat nilai K menggunakan 3, 7, 9 dan 13 dihasilkan kecocokan sebanyak 27 baris dari total 30 atau sekitar 90%. Dari percobaan tersebut dapat disimpulkan bahwa penggunaan nilai K = 3, 7, 9 dan 13  adalah sama untuk mendapatkan tingkat persentase keberhasilan penentuan kelas dari sebuah sampel baru yang masuk.